In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from zigzag import *
import datetime

In [2]:
df = pd.read_csv('data\EURUSD.csv').iloc[:, 1:]
df['datetime'] = pd.to_datetime(df.loc[:,df.columns[2]])
df.head()

,ticker,per,datetime,open,hight,low,close,volume
0,SPFB.ED,1,2009-06-15 19:52:00,1.3773,1.3773,1.3772,1.3772,4
1,SPFB.ED,1,2009-06-15 19:53:00,1.3766,1.3766,1.3760,1.3763,257
2,SPFB.ED,1,2009-06-15 19:56:00,1.3770,1.3770,1.3770,1.3770,100
3,SPFB.ED,1,2009-06-15 19:58:00,1.3769,1.3773,1.3769,1.3773,101
4,SPFB.ED,1,2009-06-15 20:00:00,1.3769,1.3769,1.3769,1.3769,6


In [3]:
df_new = df[['datetime', 'close']]
df_new.set_index(df_new.columns[0], inplace=True)
df_new.head()

,close
datetime,
2009-06-15 19:52:00,1.3772
2009-06-15 19:53:00,1.3763
2009-06-15 19:56:00,1.3770
2009-06-15 19:58:00,1.3773
2009-06-15 20:00:00,1.3769


In [4]:
def showPlot(low_lim, high_lim, asset = pd.DataFrame()):    
    col = asset.columns.values
    fig, ax = plt.subplots(figsize=(20, 10))
    
    indicator_asset = asset[asset[col[1]] != 0]
    
    ax.plot(asset[col[0]], color='red', linewidth = 1)
    ax.plot(indicator_asset[col[0]], linewidth = 2, linestyle = '--')
    
    ax.set_xlim((low_lim, high_lim))
    
    asset_down = asset[asset[asset.columns[1]] == -1][asset.columns[0]]
    asset_up = asset[asset[asset.columns[1]] == 1][asset.columns[0]]
    
    ax.scatter(asset_down.index, asset_down.values, color='r')
    ax.scatter(asset_up.index, asset_up.values, color='g')

In [5]:
h = 0.0131
indicators = peak_valley_pivots(df_new.close.values, h, -h)
df_indic = df_new.assign(indicator = indicators)

In [6]:
from ipywidgets import interact_manual
from ipywidgets import widgets
from IPython.display import clear_output
from ipywidgets import interact
@interact_manual(h=(0.0001, 0.1, 0.0001), 
                 start_date = widgets.DatePicker(value=(df_new.index[0].date() - datetime.timedelta(days=100))),
                 end_date = widgets.DatePicker(value=(df_new.index[-1].date() + datetime.timedelta(days=100))))
def plot_zigzag(h, start_date, end_date):
    indicators = peak_valley_pivots(df_new.close.values, h, -h)
    df_indic = df_new.assign(indicator = indicators)
    showPlot(start_date, end_date, df_indic)
    
    print(df_indic.indicator[df_indic.indicator != 0].value_counts())
    print(f"h: {h}, total count pivots: {df_indic.indicator[df_indic.indicator != 0].shape[0]}")
    delta = end_date - start_date
    print(f'Between {start_date} and {end_date} ==> {delta.days} days')

interactive(children=(FloatSlider(value=0.05, description='h', max=0.1, min=0.0001, step=0.0001), DatePicker(v…